# 4. Feature extraction

In [1]:
import napari
import pyclesperanto_prototype as cle
import apoc
import os
from skimage.io import imread, imsave
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from napari_skimage_regionprops import regionprops_table
from napari_simpleitk_image_processing import label_statistics

cle.select_device("Intel(R) Iris(TM) Plus Graphics 640")

<Intel(R) Iris(TM) Plus Graphics 640 on Platform: Apple (2 refs)>

In [2]:
image_folder = '../crystal data/1. normalized images/'
labels_folder = '../crystal data/3. excluded small labels/'

In [3]:
file_list = sorted(os.listdir(image_folder))
file_list

['img01_20220429_ET358_56hpf_1.tif',
 'img02_20220429_ET358_56hpf_2.tif',
 'img03_20220429_ET358_56hpf_3.tif',
 'img05_20220513_ET358_56hpf_1.tif',
 'img06_20220513_ET358_56hpf_2.tif',
 'img08_20220513_ET358_56hpf_4.tif',
 'img10_20220513_ET358_56hpf_6.tif',
 'img12_20220513_ET358_56hpf_8.tif',
 'img13_20220513_ET358_56hpf_9.tif',
 'img14_20220520_hsp-mKate-DN-dynactin_56hpf_1.tif',
 'img15_20220520_hsp-mKate-DN-dynactin_56hpf_2.tif',
 'img17_20220520_hsp-mKate-DN-dynactin_56hpf_4.tif',
 'img18_20220520_hsp-mKate-DN-dynactin_56hpf_5.tif']

In [4]:
def analyze_image(filename):
    image = imread(image_folder + filename)
    label = imread(labels_folder + filename)

    # measure in normalized image
    df_skimage = pd.DataFrame(regionprops_table(image , label, size = True, intensity = True, perimeter = True, shape = True))
    df_skimage["aspect_ratio"] = df_skimage["major_axis_length"]/df_skimage["minor_axis_length"]
    df_simpleitk = pd.DataFrame(label_statistics(image, label, size = True, intensity = True, perimeter = True, shape = True))
    DF = pd.merge(df_skimage, df_simpleitk, on = "label", suffixes = ('_skimage', '_simpleitk'))
    
    #return BIG_DF
    DF["filename"] = filename
    return DF

In [5]:
df_all_images = pd.DataFrame()

In [6]:
for i, filename in enumerate(file_list):
    df_one_image = analyze_image(filename)
    df_all_images = pd.concat([df_all_images, df_one_image], ignore_index = True)
df_all_images

,label,area,bbox_area,convex_area,equivalent_diameter,max_intensity,mean_intensity,min_intensity,perimeter_skimage,perimeter_crofton,...,equivalent_ellipsoid_diameter_0,equivalent_ellipsoid_diameter_1,equivalent_spherical_perimeter,equivalent_spherical_radius,number_of_pixels,number_of_pixels_on_border,perimeter_simpleitk,perimeter_on_border,perimeter_on_border_ratio,filename
0,1,1031,1416,1124,36.231340,6.700000,2.412512,0.000000,148.225397,143.208005,...,21.820379,60.159814,113.824110,18.115670,1031,0,143.208005,0.0,0.000000,img01_20220429_ET358_56hpf_1.tif
1,2,716,1134,855,30.193369,3.000000,1.067877,0.000000,173.574639,170.118460,...,15.432845,59.071383,94.855265,15.096684,716,0,170.118460,0.0,0.000000,img01_20220429_ET358_56hpf_1.tif
2,3,2629,5313,3558,57.856259,11.000000,2.758273,0.000000,304.462987,291.953265,...,53.928848,62.069688,181.760800,28.928130,2629,0,291.953265,0.0,0.000000,img01_20220429_ET358_56hpf_1.tif
3,4,51,60,52,8.058239,2.600000,1.239216,0.200000,25.656854,27.005740,...,6.032885,10.763543,25.315705,4.029120,51,10,27.005740,10.0,0.370292,img01_20220429_ET358_56hpf_1.tif
4,5,61,112,84,8.812923,2.600000,0.929508,0.100000,33.520815,39.072567,...,5.964736,13.021131,27.686614,4.406462,61,0,39.072567,0.0,0.000000,img01_20220429_ET358_56hpf_1.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,18,94,153,115,10.940042,4.555556,0.990544,0.111111,44.798990,45.546322,...,7.391787,16.191553,34.369155,5.470021,94,0,45.546322,0.0,0.000000,img18_20220520_hsp-mKate-DN-dynactin_56hpf_5.tif
249,19,280,595,387,18.881395,2.777778,0.936111,0.000000,111.497475,111.854667,...,11.575331,30.798867,59.317651,9.440697,280,0,111.854667,0.0,0.000000,img18_20220520_hsp-mKate-DN-dynactin_56hpf_5.tif
250,20,89,207,115,10.645108,2.555556,1.018727,0.111111,50.384776,51.790039,...,4.907889,23.089013,33.442592,5.322554,89,0,51.790039,0.0,0.000000,img18_20220520_hsp-mKate-DN-dynactin_56hpf_5.tif
251,21,358,868,573,21.349936,3.444444,1.021105,0.000000,136.124892,136.543681,...,16.424966,27.751641,67.072801,10.674968,358,0,136.543681,0.0,0.000000,img18_20220520_hsp-mKate-DN-dynactin_56hpf_5.tif


In [7]:
df_all_images.to_csv("Measurements_already_on_excluded_labels.csv")